# CONTENT BASED  RECOMMENDATION SYSTEM FOR TELUGU MOVIES

Recommender System is a system that seeks to predict or filter preferences according to the user’s choices. Recommender systems are utilized in a variety of areas including movies, music, news, books, research articles, search queries, social tags, and products in general.Recommender systems produce a list of recommendations in any of the two ways

Collaborative filtering: Collaborative filtering approaches build a model from user’s past behavior (i.e. items purchased or searched by the user) as well as similar decisions made by other users. This model is then used to predict items (or ratings for items) that user may have an interest in.

Content-based filtering: Content-based filtering approaches uses a series of discrete characteristics of an item in order to recommend additional items with similar properties. Content-based filtering methods are totally based on a description of the item and a profile of the user’s preferences. It recommends items based on user’s past preferences.


This model is trained to recommend telugu movies to users

Importing the necessary libraries.

In [16]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
df=pd.read_csv("telugu_movies.csv")
(df)

,Year,Month,Title,Director,Cast,Genre
0,1992,NaN,Aa Okkati Adakku,E. V. V. Satyanarayana,"Rajendra Prasad, Rambha, Rao Gopal Rao",NaN
1,1992,NaN,Pelli Neeku Sukham Naaku,P. N. Ramchandra Rao,"Naresh, Divya Vani, Brahmanandam",NaN
2,1992,NaN,Pellaniki Premalekha Priyuraliki Subhalekha,Relangi Narasimha Rao,"Rajendra Prasad, Nandini, Babu Mohan, Brahmana...",NaN
3,1992,NaN,Pellante Noorella Panta,Muthyala Subbaiah,"S. P. Balasubrahmanyam, Srinivasa Varma, Aishw...",NaN
4,1992,NaN,Pellam Chebithe Vinali,Kodi Ramakrishna,"Harish Kumar, Meena, Meka Srikanth",NaN
...,...,...,...,...,...,...
2052,2019,JUN,Vajra Kavachadhara Govinda,Arun Pawar,"Saptagiri, Vaibhavi Joshi, Veda Sastry, Jabard...",Drama
2053,2019,JUN,Viswamitra,Raajkiran,"Prasanna, Nanditha Raj, Ashutosh Rana, Satyam ...",Thriller
2054,2019,JUN,1st Rank Raju,Naresh Kumar H. N.,"Chethan Maddineni, Kashish Vohra, Vennela Kishore",Comedy
2055,2019,MAY,Abhinetri 2,A. L. Vijay,"Prabhu Deva, Tamannaah, Nandita Swetha, Dimple...",Comedy Horror


Preprocessing of data by filling missing values.We have selected movies from the year 2015 for our recommendation system

In [17]:
m1=df["Year"]>=2015
df=df[m1]
print(len(df))
l1=np.arange(len(df))
# removing values
print(len(df['Genre']))
df['Cast']=df['Cast'].dropna()
df['Director']=df['Director'].dropna()
df['Genre']=df['Genre'].dropna()
df['Genre']=df['Genre'].fillna('')
df['Cast']=df['Cast'].fillna('')
df['Director']=df['Director'].fillna('')
df.index=l1


692
692


Data cleaning is done so as to remove the punctuation marks and and  making the data more oragnised to be trained. 

In [18]:
l6=[]
for i in range(0,len(df)):
    st=df["Cast"].values[i].split(',')
    z=len(st)
    if z>4:
        min=2
    else:
        min=z
    st2=""
    for j in range(0,min):
        st2=st2+" "+(st[j].replace(' ',''))
        
    l6.append(st2)
l6
df["Cast"]=l6
df.head()

,Year,Month,Title,Director,Cast,Genre
0,2015,SEP,Courier Boy Kalyan,Premsai,Nithiin YamiGautam,Romance
1,2015,SEP,Subramanyam For Sale,Harish Shankar,SaiDharamTej ReginaCassandra AdahSharma NagaBabu,Romance
2,2015,SEP,Bale Bale Magadivoy,Maruthi,Nani LavanyaTripathi,Comedy Romance
3,2015,SEP,Dynamite,Deva Katta,ManchuVishnu PranithaSubhash J.D.Chakravarthy,action
4,2015,AUG,Best Actors,Arun Pawar,Nandu(AnandKrishna) Madhurima,Rom Com


COMBINING THE SELECTED FEATURES TO BULID THE RECOMMENDATION SYSTEM


In [31]:
def combine(row):
    try:
       
        return row["Director"]+" "+row["Cast"]+" "+row["Genre"]
       
        return row["Cast"]
    except:
        print(row)
df.head()

,Year,Month,Title,Director,Cast,Genre,features
0,2015,SEP,Courier Boy Kalyan,Premsai,Nithiin YamiGautam,Romance,Premsai Nithiin YamiGautam Romance
1,2015,SEP,Subramanyam For Sale,Harish Shankar,SaiDharamTej ReginaCassandra AdahSharma NagaBabu,Romance,Harish Shankar SaiDharamTej ReginaCassandra A...
2,2015,SEP,Bale Bale Magadivoy,Maruthi,Nani LavanyaTripathi,Comedy Romance,Maruthi Nani LavanyaTripathi Comedy Romance
3,2015,SEP,Dynamite,Deva Katta,ManchuVishnu PranithaSubhash J.D.Chakravarthy,action,Deva Katta ManchuVishnu PranithaSubhash J.D.C...
4,2015,AUG,Best Actors,Arun Pawar,Nandu(AnandKrishna) Madhurima,Rom Com,Arun Pawar Nandu(AnandKrishna) Madhurima Rom Com


In [32]:
df["features"]=df.apply(combine,axis=1)

In [33]:
cv=CountVectorizer()
count_matrix=cv.fit_transform(df["features"])

In [34]:
cosine_sim=cosine_similarity(count_matrix)
print((cosine_sim))
df.head()

[[1.         0.18898224 0.2236068  ... 0.         0.         0.        ]
 [0.18898224 1.         0.16903085 ... 0.         0.         0.        ]
 [0.2236068  0.16903085 1.         ... 0.18257419 0.16903085 0.        ]
 ...
 [0.         0.         0.18257419 ... 1.         0.15430335 0.        ]
 [0.         0.         0.16903085 ... 0.15430335 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


,Year,Month,Title,Director,Cast,Genre,features
0,2015,SEP,Courier Boy Kalyan,Premsai,Nithiin YamiGautam,Romance,Premsai Nithiin YamiGautam Romance
1,2015,SEP,Subramanyam For Sale,Harish Shankar,SaiDharamTej ReginaCassandra AdahSharma NagaBabu,Romance,Harish Shankar SaiDharamTej ReginaCassandra A...
2,2015,SEP,Bale Bale Magadivoy,Maruthi,Nani LavanyaTripathi,Comedy Romance,Maruthi Nani LavanyaTripathi Comedy Romance
3,2015,SEP,Dynamite,Deva Katta,ManchuVishnu PranithaSubhash J.D.Chakravarthy,action,Deva Katta ManchuVishnu PranithaSubhash J.D.C...
4,2015,AUG,Best Actors,Arun Pawar,Nandu(AnandKrishna) Madhurima,Rom Com,Arun Pawar Nandu(AnandKrishna) Madhurima Rom Com


In [35]:
def get_title_from_index(index):
    return df[df.index==index]["Title"].values[0]
def get_index_from_title(title):
    
        return df[df.Title==title].index[0]
   

In [36]:
# df[df.Title=="Aagadu"].index[0]

In [37]:
movie_user_likes="Baahubali: The Conclusion"
movie_index=get_index_from_title(movie_user_likes)
#print(list(enumerate(cosine_sim[movie_index])))# here enumerate functions give the tupple of the form (index, value)
similar_movies=list(enumerate(cosine_sim[movie_index])) 
print(similar_movies)

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.6666666666666669), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.36514837167011077), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.1666666666666667), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (71, 0.0), (72, 0.0), (73, 0.0), (74, 0.0), (75, 0.0), (76, 0.0), (77, 0.0), (78, 0.0), (79, 0.0), (80, 0.0), (81, 0.0), (82, 0.0), (83, 0.0), (84, 0.0), (85, 0.0), (86, 0.0), (87, 0

In [38]:
df.tail(30)

,Year,Month,Title,Director,Cast,Genre,features
662,2019,JUL,Marketlo Prajaswamyam,R. Narayana Murthy,R.NarayanaMurthy Madhavi,Drama,R. Narayana Murthy R.NarayanaMurthy Madhavi D...
663,2019,JUL,Rajdhoot,Arjun Gunnala,MeghamshSrihari Nakshatra,Action,Arjun Gunnala MeghamshSrihari Nakshatra Action
664,2019,JUL,iSmart Shankar,Puri Jagannadh,RamPothineni NidhhiAgerwal,Sci- Fi Action,Puri Jagannadh RamPothineni NidhhiAgerwal Sci...
665,2019,JUL,Dear Comrade,Bharat Kamma,VijayDevarakonda RashmikaMandanna ShrutiRamac...,"Action, Romance, Drama",Bharat Kamma VijayDevarakonda RashmikaMandann...
666,2019,JUL,Nenu Lenu,Ram Kumar A.S.K,HarshithKoditham SriPadma,Romance,Ram Kumar A.S.K HarshithKoditham SriPadma Rom...
667,2019,JUL,Bailampudi,Anil Raj,HarishVinay TanishqTiwari,Drama,Anil Raj HarishVinay TanishqTiwari Drama
668,2019,JUN,Voter,G. S. Karthik,ManchuVishnu Surabhi,Action drama,G. S. Karthik ManchuVishnu Surabhi Action drama
669,2019,JUN,Special - The Story of a Mind Reader,Vastav,Ajay Ranga Akshatha JabardasthAppaRao,Thriller drama,Vastav Ajay Ranga Akshatha JabardasthAppaRao ...
670,2019,JUN,Mallesham,Raj. R,PriyadarshiPulikonda AnanyaNagalla Chakrapani...,Drama,Raj. R PriyadarshiPulikonda AnanyaNagalla Cha...
671,2019,JUN,Agent Sai Srinivasa Athreya,Swaroop RSJ,NaveenPolishetty ShruthiSharma DarbhaAppajiAm...,Comedy Thriller,Swaroop RSJ NaveenPolishetty ShruthiSharma Da...


In [39]:
df.iloc[0]['Cast']

' Nithiin YamiGautam'

In [40]:
# sim_movies are in tupple so we are sorting it based
# on 2nd data point.

sorted_similar_movies=sorted(similar_movies,key= lambda x:x[1],reverse=True)[1:]# here the 1st element will be the same movie 
# bcoz its correlated 100% so 1st element is left out whil sorting and the also while assigning to similar movies the seen
# movie must be dropped so [1:]is placed at the last. reverse implies descedning order. here key is a function
print(sorted_similar_movies)

[(8, 0.6666666666666669), (31, 0.36514837167011077), (476, 0.18257418583505539), (551, 0.18257418583505539), (41, 0.1666666666666667), (150, 0.1666666666666667), (466, 0.1666666666666667), (560, 0.1666666666666667), (328, 0.1543033499620919), (319, 0.14433756729740646), (410, 0.12909944487358058), (444, 0.12909944487358058), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), 

THE MODEL RECOMMENDS 4 MOVIES TO THE USER

In [41]:
i=0
print("top 4 similar movies to"+movie_user_likes+" are :\n")
for ele in sorted_similar_movies:
    
    print(get_title_from_index(ele[0]))
    i=i+1
    if i>4:
        break

top 4 similar movies toBaahubali: The Conclusion are :

Baahubali: The Beginning
Rudhramadevi
Bhaagamathie
Saaho
Kanche
